In [17]:
# Libraries
import snowflake.connector
import pandas as pd
import time
import numpy as np
from datetime import date
from datetime import datetime
import os
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch

# Snowflake Credentials
uname=""
pwd=""
url=""
wh=""

In [18]:
# Establish snowflake connection 
def sf_conn(user, pwd, url, wh=''):
    cnx = snowflake.connector.connect(
        user=user,
        password=pwd,
        account=url,
        warehouse=wh
        )
    return cnx.cursor()

In [19]:
# Execute SQL query and return in a dataframe !!! Do replace the Database name & Schema Name in SQL queries !!!
def exec_query(query, user, pwd,url):
    cur = sf_conn(user,pwd,url)
    start_time = time.time()
    getData = cur.execute(query).fetchall()
    column_names = [i[0] for i in cur.description]
    cur.close()
    #print("Time :", ((time.time()-start_time))/60)
    return(pd.DataFrame(getData, columns=column_names))

In [73]:
# Execute & ensure that the data is loaded correctly and is present in the table
query = """
      SELECT * FROM <DB_NAME>.<SCHEMA_NAME>.GOOGLE_NEWS;
    """
tempdf = exec_query(query, uname, pwd,url)
tempdf.head(3)

,CONTACT_DOMAIN,DATE_TIME_NEWS,SNIPPET,TITLE,URL
0,washingtontechnology.com,3 days ago,The company makes this executive team addition...,Easy Dynamics hires new finance chief,https://washingtontechnology.com/companies/202...
1,ft.com,1 day ago,US financial technology start-up Plaid has hir...,US fintech Plaid hires first chief financial o...,https://www.ft.com/content/e818d6aa-9a32-4874-...
2,yahoo.com,1 day ago,Fintech startup Plaid has hired former Expedia...,Plaid taps former Expedia exec as its new chie...,https://techcrunch.com/2023/10/18/plaid-taps-f...


In [21]:
# Vector search works on embeddings, instantiate embedding model of your choice
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-small-en")
model = AutoModel.from_pretrained("BAAI/bge-small-en")

In [22]:
# From the dataframe which has the data from table GOOGLE_NEWS, 
# choose your column for which you want to create vector embeddings.
# In this case we are choosing Title for which we will create embeddings
# and store it under column Vector

# Generate embeddings for each text sample (TITLE) 
embeddings = []
for text in tempdf['TITLE']:
    encoded_input = tokenizer(text.lower(), padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings.append(model_output.last_hidden_state.mean(dim=1).squeeze().tolist())

# Convert embeddings to a numpy array
embeddings_arr = np.array(embeddings)

tempdf['vector']=''
for index, row in tempdf.iterrows():
    tempdf.at[index, 'vector'] = embeddings_arr[index].tolist()

In [39]:
%%time

# Now insert the rows from the df in the Vector table created in Snowflake
cur = sf_conn(uname,pwd,url)

sql = """INSERT INTO <DB_NAME>.<SCHEMA_NAME>.GOOGLE_NEWS_EMBEDDINGS
SELECT '{CONTACT_DOMAIN}','{DATE_TIME_NEWS}', '{SNIPPET}',
 '{TITLE}', '{URL}',
 parse_json('{vector}')
"""
# Insert rows to Snowflake vector table

for i, r in tempdf.iterrows():
    try:
        r['TITLE'] = r['TITLE'].lower().replace("'", "")
        r['SNIPPET'] = r['SNIPPET'].replace("'", "")
        cur.execute(sql.format(**dict(r)))
    except Exception as e:
        print(" --> Error at index:", i)
        # print(r)
        print(" Error:", str(e))
        continue

CPU times: user 164 ms, sys: 81.3 ms, total: 246 ms
Wall time: 7.33 s


In [45]:
# Ensure the data is loaded in the vector table 
# and the vector column has vectors 
query = """
      SELECT * FROM <DB_NAME>.<SCHEMA_NAME>.GOOGLE_NEWS_EMBEDDINGS;
    """
tempdf = exec_query(query, uname, pwd,url)
tempdf.head(2)

,CONTACT_DOMAIN,DATE_TIME_NEWS,SNIPPET,TITLE,URL,VECTOR
0,washingtontechnology.com,3 days ago,The company makes this executive team addition...,easy dynamics hires new finance chief,https://washingtontechnology.com/companies/202...,"[\n -0.5620648264884949,\n -0.34428820013999..."
1,ft.com,1 day ago,US financial technology start-up Plaid has hir...,us fintech plaid hires first chief financial o...,https://www.ft.com/content/e818d6aa-9a32-4874-...,"[\n -0.1221204400062561,\n -0.01620235480368..."


In [46]:
tempdf.shape

(20, 6)

In [43]:
# Function to convert user query to vector embeddings
def get_query_embeddings(query, tokenizer=tokenizer, model=model):
    input_list = []
    input_list.append(query)
    embeddings_q = []
    for text in input_list:
        encoded_input = tokenizer(text.lower(), padding=True, truncation=True, max_length=512, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**encoded_input)
        embeddings_q.append(model_output.last_hidden_state.mean(dim=1).squeeze().tolist())

    # Convert embeddings to a numpy array
    embeddings_arr_q = np.array(embeddings_q)
    return embeddings_arr_q

In [47]:
# Function to execute Python UDF that will calculate cosine distance
# between user query and vectors stored inside the vector table

def get_sim_from_sf(query_emb, uname=uname, pwd=pwd,url=url):
    query = """
      SELECT distinct title, url, 
        PY_COSINE_SIMILARITY(vector, {}) as cosine_sim
 FROM <DB_NAME>.<SCHEMA_NAME>.GOOGLE_NEWS_EMBEDDINGS
 ORDER BY  cosine_sim desc; 
    """.format(query_emb)
    uname=uname
    pwd=pwd
    url=url
    return exec_query(query, uname, pwd,url)


In [74]:
# Function that returns response to user query
def do_vector_search(query):
    embeddings_arr_q = get_query_embeddings(query) 
    return get_sim_from_sf(embeddings_arr_q[0].tolist())
    


In [76]:
# Lets query Snowflake 
query = 'Who is hiring chief financial officer?'
df_vector_search_response = do_vector_search(query)
df_vector_search_response.head(3)

,TITLE,URL,COSINE_SIM
0,hiring a chief financial officer,https://a16z.com/hiring-a-chief-financial-offi...,0.967084
1,spothero hires new chief financial officer,https://www.businesswire.com/news/home/2023101...,0.871449
2,community roundup: magruder hires chief financ...,https://www.thenews-messenger.com/story/news/l...,0.833398


In [80]:
# Lets query Snowflake 
query = 'Who hired technology officer?'
df_vector_search_response = do_vector_search(query)
df_vector_search_response.head(3)

,TITLE,URL,COSINE_SIM
0,crossroads equipment lease and finance hires b...,https://www.monitordaily.com/news-posts/crossr...,0.767865
1,hiring a chief financial officer,https://a16z.com/hiring-a-chief-financial-offi...,0.763271
2,us fintech plaid hires first chief financial o...,https://www.ft.com/content/e818d6aa-9a32-4874-...,0.740950


In [82]:
print(df_vector_search_response.iloc[0].TITLE)

crossroads equipment lease and finance hires balaraman as chief technology 
officer
